<a href="https://colab.research.google.com/github/MikolajKita/Neural-Networks/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip p4.zip

Archive:  p4.zip
  inflating: test_no_target.pkl      
  inflating: train.pkl               
  inflating: treШЖ_zadania.txt     


In [49]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [50]:
import pickle


with open('train.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [51]:
device = torch.device('cuda') # 
#device = torch.device('cpu')
device

device(type='cuda')

In [52]:
torch.manual_seed(42)
if torch.cuda.is_available(): 
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    
# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

In [53]:
torch.set_default_dtype(torch.float64)

In [54]:
type(dataset)

list

In [55]:
dataset[642]

(array([ -1.,  -1., 145.,   5.,  88.,  13.,  47.,  47.,  12., 112.,  37.,
        132.,  12.,  77.,  92., 156.,  12., 152.,  92.,  12.,  47.,  13.,
         47.,  47.,  12.,  78., 119.,  78.,  12.,   8.,  88.,  47.,  78.,
         12.,  92.,   5.,  88.,  13.,  47.,  47.,  12., 112.,  37., 132.,
         12.,  77.,  92., 156.,  12., 152.,  92.,  12.,  47.,  13.,  47.,
         47.,  12.,  78., 119.,  78.,  12.,   8.,  88.,  47.,  78., 131.,
          5.,  76.,  79.,  12., 159., 185., 124.,  32.,  39., 190.,  12.,
        159., 124., 124., 124.,  79.,  76.,  12.,  44., 157.,  44.,  40.,
         44.,  88.,  30.,  88.,  88., 172.,  92.,  44., 191., 172., 172.,
        158.,  60., 120.,  60.,  37.,  37.,  60.,  60.,  20.,  60.,  60.,
         60.,  44.,  44.,  33.,  33.,  47.,  12., 172.,  92.,  78.,  12.,
         69.,  92.,  12.,  69., 159.,  47., 172., 152.,  74.,  12., 159.,
          0.,  47.,  47.,  47.,  73., 114., 159., 132., 125.,  78.,  12.,
         13.,  12., 152.,  12.,  12., 

In [56]:
len(dataset)

2939

In [57]:
dataset[642][1]



0

In [58]:
#{0: 'bach', 1: 'beethoven', 2: 'debussy', 3: 'scarlatti', 4: 'victoria'}.

class_bach_0 = 0
class_beethoven_1 = 0
class_debussy_2 = 0
class_scarlatti_3 = 0
class_victoria_4 = 0


In [59]:
for i in range (0,len(dataset)):
  if(dataset[i][1] == 0):
    class_bach_0 += 1
  if(dataset[i][1] == 1):
    class_beethoven_1 += 1
  if(dataset[i][1] == 2):
    class_debussy_2 += 1
  if(dataset[i][1] == 3):
    class_scarlatti_3 += 1
  if(dataset[i][1] == 4):
    class_victoria_4 += 1
print(class_bach_0, class_beethoven_1, class_debussy_2, class_scarlatti_3, class_victoria_4)

In [60]:
class_bach_0


1630

In [61]:
class_beethoven_1


478

In [62]:
class_debussy_2


154

In [63]:
class_scarlatti_3


441

In [64]:
class_victoria_4

236

In [65]:
oversampled_dataset = dataset

In [66]:
len(oversampled_dataset)

2939

In [67]:
for i in range (0,len(dataset)):
  if(dataset[i][1] == 1):
    class_beethoven_1 += 1
    oversampled_dataset.append(dataset[i])
  if(dataset[i][1] == 2):
    class_debussy_2 += 1
    oversampled_dataset.append(dataset[i])
  if(dataset[i][1] == 3):
    class_scarlatti_3 += 1
    oversampled_dataset.append(dataset[i])
  if(dataset[i][1] == 4):
    class_victoria_4 += 1
    oversampled_dataset.append(dataset[i])

In [68]:
oversampled_dataset[2940]

(array([144.,  13., 191., 144.,  44., 156.,  40., 156.,  40.,  13.,  13.,
        156.,  40., 156.,  92.,   8., 156., 156., 159.,  12.,  12.,  92.,
         92., 125.,  12.,  88.,  80.,  41.,  78.,  12.,  92.,  41.,  60.,
          0.,  64.,  33.,  40., 144., 144.,  30., 132.,  37.,  32.,  44.,
         30.,  44.,  44., 177.,  44., 156.,  64., 156.,  30., 144.,  30.,
         30.,  44.,  44.,  44.,  30.,  44.,  44.,  73.,  44.,  44.,  30.,
        156., 151.,  68.,  79.,  77., 156., 156.,  44.,  68.,  79.,  77.,
        156., 156., 144., 144.,  13., 191., 144.,  44., 156.,  40., 156.,
         40.,  13.,  13., 156.,  40., 156.,  92.,   8., 156., 156., 159.,
         12.,  12.,  12.,  28., 125.,  12.,  88.,  80.,  41.,  78.,  12.,
         92.,  41.,  78.,  64.,  64.,  64.,  33., 144., 144., 144.,  30.,
        132.,  37.,  32.,  44.,  30.,  44.,  44., 177.,  44., 156.,  64.,
        156.,  30., 144.,  30.,  30.,  37.,  37.,  44.,  30.,  44.,  44.,
         73.,  44.,  44.,  30., 156., 

In [69]:
from torch.utils.data import Dataset, DataLoader
class VariableLenDataset(Dataset):
    def __init__(self, in_data):
        self.data = [(x, y) for x, y in in_data]    
        
    def __len__(self):
        return len(self.data)

    def max(self):
        max = 0
        for data, target in self.data:
            if(len(data) > max):
              max = len(data)
        return max
          

    def __getitem__(self, idx):
        seq, target = self.data[idx]
        return seq, target

In [70]:
train_part = int(0.8 * len(dataset))
train_size = int(0.8 * train_part)
validation_size = train_part - train_size
test_size = len(dataset) - train_part
train_set, validation_set, test_set = torch.utils.data.random_split(oversampled_dataset, [train_size, validation_size, test_size])

In [71]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
def collate_fn(batch, pad_value=0):
    """
       data: is a list of tuples with (example, label, length)
             where 'example' is a tensor of arbitrary shape
             and label/length are scalars
    """
    #print(batch[1])
    #inputs = [torch.tensor(d['tokenized_input']) for d in data] #(3)
   # labels = [d['label'] for d in data]
    seq, target = zip(*batch)
    inputs = [torch.tensor(x) for x in seq] #(3)
    labels = torch.tensor(target)
    seq_pad = pad_sequence(inputs, batch_first=True)
    return seq_pad, labels

In [72]:
train_loader = DataLoader(train_set, batch_size=50, shuffle=True, drop_last = True, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=50, shuffle=False, drop_last = True, collate_fn=collate_fn)
validation_loader = DataLoader(validation_set, batch_size=50, shuffle=False, drop_last = True, collate_fn=collate_fn)

In [73]:
len(train_set)

2718

In [74]:
len(validation_set)

680

In [75]:
len(test_set)

850

In [76]:
train_set[0]

(array([ -1.,  -1.,  -1.,  -1.,   0., 148.,   1., 180., 146.,  73.,  69.,
         92.,  60.,  47.,  12.,  12., 148.,  20.,  68., 132., 159.,  68.,
        185., 124., 124.,  37.,  79.,  12.,  28.,  28.,  28.,  28.,  92.,
         12.,  47.,  92.,  88., 159.,  41.,  12.,  12.,  47., 159.,  92.,
         28.,  28.,  28.,  28.,  28.,  47.,  28.,  93.,  28.,  13.,  45.,
        125., 157.,  77.,  78.,  47., 125., 157.,  47.,  93.,  12.,  12.,
         47.,  47.,  28.,  12., 159.,  13.,  28.,  28.,  28.,  28.,  12.,
         73., 124.,   5., 185., 159., 124., 124.,  12.,  12.,  45.,  15.,
        124., 159., 124., 185.,  47.,  20.,  47.,  12., 119.,  12.,  28.,
         47.,  12.,  13.,  28.,  28.,  28.,  28.,  47.,  13.,  28., 159.,
         28.,  60.,  28., 159.,  12.,  12., 159., 124., 124., 159.,  92.,
         76.,  76.,  12., 106.,  76.,  76.,  76.,  79., 159., 190.,  79.,
        190.,  62.,  76.,  60., 124.,  30.,  44., 111., 156.,  12.,  79.,
        124.,  12.,  45., 111., 156., 

In [77]:
train_features, train_labels = next(iter(train_loader))

In [78]:
train_features[0].size()

torch.Size([5299])

In [79]:
train_features[1].size()

torch.Size([5299])

In [80]:
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, out_size)
        
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, state
    
    def forward(self, x, hidden):
        x = torch.transpose(x, 0, 1)
        all_outputs, hidden = self.lstm(x, hidden)
        out = all_outputs[-1] # We are interested only in the last output
        x = self.fc(out)
        return x, hidden
    
model = LSTMRegressor(input_size=1, hidden_size = 20, num_layers = 3, out_size = 5).to(device)
model

LSTMRegressor(
  (lstm): LSTM(1, 20, num_layers=3)
  (fc): Linear(in_features=20, out_features=5, bias=True)
)

In [81]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fun = nn.CrossEntropyLoss()

num_epochs = 20
training_loss = 0
validation_loss = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    for i, data in enumerate(train_loader, 0):
      #print(i, data)
      #print(i, data)

      inputs, labels = data
      inputs = inputs.to(device).unsqueeze(2)
      #inputs = [seq.to(device) for seq in inputs]
      labels = labels.to(device)
      #print(inputs)
      hidden, state = model.init_hidden(inputs.size(0))
      hidden, state = hidden.to(device), state.to(device)
      preds, last_hidden = model(inputs, (hidden,state))
      optimizer.zero_grad() 
      loss = loss_fun(preds, labels)
      loss.backward()
      optimizer.step()
      training_loss += loss.item()

    model.eval()
    with torch.no_grad():
      for i, data in enumerate(validation_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device).unsqueeze(2)
        labels = labels.to(device)
        #hidden, state = hidden.to(device), state.to(device)
        preds, last_hidden = model(inputs, (hidden,state))
        loss = loss_fun(preds, labels)
      
        validation_loss += loss.item()

    if epoch % 5 == 0:
      print(f"Epoch: {epoch}, training loss: {training_loss/1:.3}, validation loss: {validation_loss/1:.3}")
      training_loss = 0
      validation_loss = 0

Epoch: 0, training loss: 81.3, validation loss: 19.0
Epoch: 5, training loss: 3.95e+02, validation loss: 94.8
Epoch: 10, training loss: 4.02e+02, validation loss: 1.04e+02
Epoch: 15, training loss: 3.97e+02, validation loss: 95.1


In [82]:
correct_pred = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
total_pred = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
correct_pred.items()

dict_items([(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)])

In [83]:
acc_sum = 0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device).unsqueeze(2)
        labels = labels.to(device)
        #hidden, state = hidden.to(device), state.to(device)
        preds, last_hidden = model(inputs, (hidden,state))
        _, predictions = torch.max(preds, 1)
        for label, prediction in zip(labels, predictions):
          if label == prediction:
              correct_pred[label.item()] += 1
          total_pred[label.item()] += 1



In [84]:
correct_pred.items()

dict_items([(0, 323), (1, 17), (2, 0), (3, 0), (4, 0)])

In [85]:
total_pred.items()

dict_items([(0, 326), (1, 197), (2, 50), (3, 184), (4, 93)])

In [86]:
acc_sum = 0
for classname, correct_count in correct_pred.items():
    print("Class: {}, correct predictions: {}, total_predictions {}".format(classname, correct_count, total_pred[classname]))
    accuracy = 100 * float(correct_count) / total_pred[classname]
    acc_sum += accuracy
    print("Accuracy for class {} is: {:.1f} %".format(classname, 
                                                   accuracy))

Class: 0, correct predictions: 323, total_predictions 326
Accuracy for class 0 is: 99.1 %
Class: 1, correct predictions: 17, total_predictions 197
Accuracy for class 1 is: 8.6 %
Class: 2, correct predictions: 0, total_predictions 50
Accuracy for class 2 is: 0.0 %
Class: 3, correct predictions: 0, total_predictions 184
Accuracy for class 3 is: 0.0 %
Class: 4, correct predictions: 0, total_predictions 93
Accuracy for class 4 is: 0.0 %


In [87]:
acc_sum

107.70919622559248

In [88]:
acc_average_all_classes = acc_sum / len(correct_pred)
acc_average_all_classes

21.541839245118496

In [89]:
model = LSTMRegressor(input_size=1, hidden_size = 20, num_layers = 2, out_size = 5).to(device)
model

LSTMRegressor(
  (lstm): LSTM(1, 20, num_layers=2)
  (fc): Linear(in_features=20, out_features=5, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fun = nn.CrossEntropyLoss()

num_epochs = 100
training_loss = 0
validation_loss = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    for i, data in enumerate(train_loader, 0):
      #print(i, data)
      #print(i, data)

      inputs, labels = data
      inputs = inputs.to(device).unsqueeze(2)
      #inputs = [seq.to(device) for seq in inputs]
      labels = labels.to(device)
      #print(inputs)
      hidden, state = model.init_hidden(inputs.size(0))
      hidden, state = hidden.to(device), state.to(device)
      preds, last_hidden = model(inputs, (hidden,state))
      optimizer.zero_grad() 
      loss = loss_fun(preds, labels)
      loss.backward()
      optimizer.step()
      training_loss += loss.item()

    model.eval()
    with torch.no_grad():
      for i, data in enumerate(validation_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device).unsqueeze(2)
        labels = labels.to(device)
        #hidden, state = hidden.to(device), state.to(device)
        preds, last_hidden = model(inputs, (hidden,state))
        loss = loss_fun(preds, labels)
      
        validation_loss += loss.item()

    if epoch % 5 == 0:
      print(f"Epoch: {epoch}, training loss: {training_loss/1:.3}, validation loss: {validation_loss/1:.3}")
      training_loss = 0
      validation_loss = 0

Epoch: 0, training loss: 76.9, validation loss: 18.7
Epoch: 5, training loss: 3.84e+02, validation loss: 93.6


In [ ]:
correct_pred = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
total_pred = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
correct_pred.items()

In [ ]:
acc_sum = 0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device).unsqueeze(2)
        labels = labels.to(device)
        #hidden, state = hidden.to(device), state.to(device)
        preds, last_hidden = model(inputs, (hidden,state))
        _, predictions = torch.max(preds, 1)
        for label, prediction in zip(labels, predictions):
          if label == prediction:
              correct_pred[label.item()] += 1
          total_pred[label.item()] += 1



In [ ]:
acc_sum = 0
for classname, correct_count in correct_pred.items():
    print("Class: {}, correct predictions: {}, total_predictions {}".format(classname, correct_count, total_pred[classname]))
    accuracy = 100 * float(correct_count) / total_pred[classname]
    acc_sum += accuracy
    print("Accuracy for class {} is: {:.1f} %".format(classname, 
                                                   accuracy))

In [ ]:
acc_average_all_classes = acc_sum / len(correct_pred)
acc_average_all_classes